## Initial Setup
### Import libraries 

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

clean_dataset = 'cleaned'
raw_dataset = 'raw'

### Check if directory exists

In [2]:
if not os.path.isdir(clean_dataset):
    try:
        os.mkdir(clean_dataset)
    except OSError as error:
        print(error)

if not os.path.isdir(raw_dataset):
    try:
        os.mkdir(raw_dataset)
    except OSError as error:
        print(error)

### Utils

In [3]:
def missing_percentage(df):
    return df.isnull().mean().round(4).mul(100).sort_values(ascending=False)

def read_raw(filename):
    return pd.read_csv(os.path.join(raw_dataset, filename))

def read_clean(filename):
    return pd.read_csv(os.path.join(clean_dataset, filename))
    
def write_clean(df:pd.DataFrame, name:str, format='%.6f'):
    df.to_csv(os.path.join(clean_dataset, name), index=False, float_format=format)

def valid_date(date):
    try:
        pd.to_datetime(date)
        return True
    except:
        return False

## Population and density
Source for [Population density](https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf_1/explore?showTable=true) and [County Populations](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/) .

In [4]:
population = read_raw('covid_county_population_usafacts.csv')
population.head()

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394


In [5]:
population.rename(
    {'countyFIPS':'FIPS', 'population':'Population'}, 
    axis=1, inplace=True
)
population = population[population['FIPS']>0][['FIPS', 'Population']]
population.head()

,FIPS,Population
1,1001,55869
2,1003,223234
3,1005,24686
4,1007,22394
5,1009,57826


In [7]:
size_density = read_raw('Average_Household_Size_and_Population_Density_-_County.csv')
size_density.rename({'FIPS Code':'FIPS', 'Name':'County'}, axis=1, inplace=True)
size_density = size_density[['FIPS', 'County', 'State', 'Area of Land (square meters)']]

In [8]:
population = size_density.merge(population, how='inner', on='FIPS')
population['Population Density'] = population['Population'] * 1e6 / population['Area of Land (square meters)']
population.drop('Area of Land (square meters)', axis=1, inplace=True)
write_clean(population, 'Population.csv')

## Google community mobility report
The Region CSVs file is collected from [Google mobility page](https://www.google.com/covid19/mobility/), then US regions are copied from the unziped folder. [Download link](https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip). The date starts from 2020-02-15 till now.

In [29]:
# do the same for complete data and create a summary
county_summaries = []
fix_column_names = {'sub_region_2':'County', 'census_fips_code':'FIPS', 'date':'Date'}
# drop unnecessary columns, these columns are either not necessary or have large missing values
drop_columns = ['place_id', 'country_region_code', 'country_region', 'sub_region_1', 'metro_area','iso_3166_2_code', 
'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'retail_and_recreation_percent_change_from_baseline'
]

for year in range(2020, 2023):
    filepath = os.path.join(raw_dataset, f'Google mobility report for US/{year}_US_Region_Mobility_Report.csv')
    df = pd.read_csv(filepath)
    df = df.drop(drop_columns, axis=1)

    df = df.rename(fix_column_names, axis=1)
    df = df[~(df['County'].isnull()| df['FIPS'].isnull())]
    df['FIPS'] = df['FIPS'].astype(int)
    df = df[df['FIPS'].isin(population['FIPS'])]
    county_summaries.append(df)

In [30]:
mobility_google = pd.concat(county_summaries, axis=0, sort=False).reset_index(drop=True)
# mobility_google = mobility_google[mobility_google['Date']>=start_date]
missing_percentage(mobility_google)

residential_percent_change_from_baseline    42.35
workplaces_percent_change_from_baseline      1.94
County                                       0.00
FIPS                                         0.00
Date                                         0.00
dtype: float64

In [31]:
mobility_google = mobility_google[['FIPS', 'Date', 'workplaces_percent_change_from_baseline']]
write_clean(mobility_google, 'Mobility google.csv')

## Trips by Distance data
Collected from [U.S. Department of Transportation Bureau of Transportation Statistics](https://data.bts.gov/Research-and-Statistics/Trips-by-Distance/w96p-f2qv). [Download link](https://data.bts.gov/api/views/w96p-f2qv/rows.csv?accessType=DOWNLOAD). From 2019/01/01 till 2022/02/05, updated regularly.

In [43]:
mobility_bts = read_raw("Trips_by_distance.csv")
mobility_bts.columns

Index(['Level', 'Date', 'State FIPS', 'State Postal Code', 'County FIPS',
       'County Name', 'Population Staying at Home',
       'Population Not Staying at Home', 'Number of Trips',
       'Number of Trips <1', 'Number of Trips 1-3', 'Number of Trips 3-5',
       'Number of Trips 5-10', 'Number of Trips 10-25',
       'Number of Trips 25-50', 'Number of Trips 50-100',
       'Number of Trips 100-250', 'Number of Trips 250-500',
       'Number of Trips >=500', 'Row ID', 'Week', 'Month'],
      dtype='object')

In [44]:
print(mobility_bts.Date.min(), mobility_bts.Date.max())

mobility_bts = mobility_bts.drop(['State FIPS', 'State Postal Code',
'Row ID', 'Week', 'Month'], axis=1)
mobility_bts = mobility_bts[mobility_bts['Level']=='County'].reset_index(drop=True)

# replace datetime format 'year/month/day' with 'year-month-day' to be consistent with others
mobility_bts['Date'] = pd.to_datetime(mobility_bts['Date'])

# reducing file size
mobility_bts = mobility_bts[mobility_bts['Date']>pd.to_datetime('2020-02-01')]

missing_percentage_county = missing_percentage(mobility_bts)
# missing_percentage_county[missing_percentage_county>0]

2019/01/01 2022/02/05


In [45]:
# mobility_bts['Total Population'] = mobility_bts['Population Staying at Home']+mobility_bts['Population Not Staying at Home']
# mobility_bts['Ratio of Population Not Staying at Home'] = mobility_bts['Population Not Staying at Home']/ mobility_bts['Total Population']
# mobility_bts['Number of Trips Per Person'] = mobility_bts['Number of Trips'] / mobility_bts['Total Population']

mobility_bts.rename({'County FIPS':'FIPS', 'County Name':'County'}, axis=1, inplace=True)
mobility_bts['FIPS'] = mobility_bts['FIPS'].astype(int)

In [46]:
selected_columns = ['FIPS', 'Date', 'Number of Trips','Population Not Staying at Home']

# fill the rest 0.49% missing values with values from previous non-null row
# missing_percentage(mobility_bts)
mobility_bts = mobility_bts[selected_columns].fillna(method='ffill')
write_clean(mobility_bts, "Mobility bts.csv")

## Covid cases
Collected cumulative covid cases from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). It is then converted to daily cases, neg daily cases are converted to zero, then dumped in the cleaned folder.

In [30]:
df = read_raw('covid_confirmed_usafacts.csv')
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,

In [28]:
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df = df[df['FIPS'].isin(population['FIPS'])]
df = df.fillna('ffill').fillna(0)

# convert cumulative cases to daily cases
date_columns = [col for col in df.columns if valid_date(col)]
df_advanced = df[date_columns].shift(periods=1, axis=1, fill_value=0)
df[date_columns] -= df_advanced[date_columns]
df.fillna(0, inplace=True)

In [29]:
df = df.melt(
    id_vars= ['FIPS'],
    var_name='Date', value_name='Cases'
).reset_index(drop=True)
df = df.fillna(0)

# some days had old covid cases fixed by adding neg values
df.loc[df['Cases']<0, 'Cases'] = 0

write_clean(df, 'Cases.csv')
df.head()

,FIPS,Date,Cases
0,1001,2020-01-22,0
1,1003,2020-01-22,0
2,1005,2020-01-22,0
3,1007,2020-01-22,0
4,1009,2020-01-22,0


In [35]:
jhu = read_raw('time_series_covid19_confirmed_US.csv')
jhu = jhu[(~jhu['FIPS'].isna()) & (jhu['FIPS']>0)]
jhu['FIPS'] = jhu['FIPS'].astype(int)

jhu.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/2

In [48]:
temp = jhu[jhu['FIPS'].isin([55039, 55059, 55073, 55079, 55139])]
temp['diff'] = temp['11/29/21'] - temp['11/28/21']
temp[['FIPS', '11/28/21', '11/29/21', 'diff']]

C:\Users\khair\AppData\Local\Temp\ipykernel_1332\3111653692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['diff'] = temp['11/29/21'] - temp['11/28/21']


,FIPS,11/28/21,11/29/21,diff
3262,55039,19901,20058,157
3272,55059,26113,26362,249
3279,55073,24451,24780,329
3283,55079,160127,160864,737
3315,55139,30589,30925,336


In [53]:
temp = df[df['countyFIPS'].isin([55039, 55059, 55073, 55079, 55139])]
temp['diff'] = temp['2021-11-29'] - temp['2021-11-28']
temp[['countyFIPS', '2021-11-28', '2021-11-29', 'diff']]

C:\Users\khair\AppData\Local\Temp\ipykernel_1332\2135905391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['diff'] = temp['2021-11-29'] - temp['2021-11-28']


,countyFIPS,2021-11-28,2021-11-29,diff
3116,55039,19902,20058,156
3126,55059,26226,26362,136
3133,55073,24610,24780,170
3137,55079,160668,160864,196
3167,55139,30791,30925,134


## Deaths
Collected cumulative covid deaths from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). It is then converted to daily deaths, neg daily deaths are converted to zero, then dumped in the cleaned folder.

In [12]:
df = read_raw('covid_deaths_usafacts.csv')
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-04-01,2022-04-02,2022-04-03,2022-04-04,2022-04-05,2022-04-06,2022-04-07,2022-04-08,2022-04-09,2022-04-10
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,211,211,211,213,213,213,213,213,213,213
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,675,675,675,675,675,676,676,677,677,677
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,97,97,97,97,97,98,98,98,98,98
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,101,101,101,101,101,101,101,101,101,101


In [13]:
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)

# drop invalid rows
df = df[~df['FIPS'].isna()]
df = df[df['FIPS']>0]
df = df.fillna(0)

# convert cumulative cases to daily 
df = df.T
head = df.iloc[0]
df = df.iloc[1:]
df.columns = head
df = df.diff()
df = df.T.reset_index()

In [14]:
df = df.melt(
    id_vars= ['FIPS'],
    var_name='Date', value_name='Deaths'
).reset_index(drop=True)

df = df.fillna(0)
# some days had old covid deaths fixed by adding neg values
df.loc[df['Deaths']<0, 'Deaths'] = 0

write_clean(df, 'Deaths.csv')

df.head()

,FIPS,Date,Deaths
0,1001,2020-01-22,0.0
1,1003,2020-01-22,0.0
2,1005,2020-01-22,0.0
3,1007,2020-01-22,0.0
4,1009,2020-01-22,0.0


## Vaccinations
From [CDC vaccinations in the US counties](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh)

In [60]:
vaccination = read_raw('COVID-19_Vaccinations_in_the_United_States_County.csv')
print(vaccination.columns)

C:\Users\khair\AppData\Local\Temp\ipykernel_1332\689704449.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(os.path.join(raw_dataset, filename))


Index(['Date', 'FIPS', 'MMWR_week', 'Recip_County', 'Recip_State',
       'Completeness_pct', 'Administered_Dose1_Recip',
       'Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_5Plus',
       'Administered_Dose1_Recip_5PlusPop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_Pct', 'Series_Complete_5Plus',
       'Series_Complete_5PlusPop_Pct', 'Series_Complete_5to17',
       'Series_Complete_5to17Pop_Pct', 'Series_Complete_12Plus',
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18Plus',
       'Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Booster_Doses',
       'Booster_Doses_Vax_Pct', 'Booster_Doses_12Plus',
       'Booster_Doses_12Plus_V

In [61]:
vaccination = vaccination[vaccination['FIPS']!='UNK']
vaccination['FIPS'] = vaccination['FIPS'].astype(int)

vaccination_columns = ['Administered_Dose1_Pop_Pct',
'Series_Complete_Pop_Pct']
vaccination = vaccination[['Date', 'FIPS'] + vaccination_columns]

# vaccination has lots of null columns, even when there should be value available
# vaccination.loc[:, vaccination_columns] = vaccination[vaccination_columns].interpolate(method='nearest')
vaccination = vaccination.fillna(0)

vaccination['Date'] = pd.to_datetime(vaccination['Date'])

write_clean(vaccination, "Vaccination.csv", None)

C:\Users\khair\AppData\Local\Temp\ipykernel_1332\2921615446.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vaccination['FIPS'] = vaccination['FIPS'].astype(int)


## Social Vulnerability Index
From [CDC](https://coronavirus-resources.esri.com/datasets/cdcarcgis::overall-svi-counties/about)

In [32]:
svi = read_raw('CDC_Social_Vulnerability_Index_2018_-_USA.csv')
selected_columns = ['FIPS'] + [col for col in svi.columns if 'RPL_THEME' in col]
write_clean(svi[selected_columns], 'Social vulnerability index.csv')

svi[selected_columns].head()

,FIPS,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,1001,0.3631,0.5810,0.5947,0.3741,0.4354
1,1003,0.2232,0.1990,0.4358,0.3359,0.2162
2,1005,0.9780,0.9153,0.8558,0.9889,0.9959
3,1007,0.7694,0.1203,0.4323,0.7189,0.6003
4,1009,0.6143,0.3187,0.5915,0.1741,0.4242


## County Health Ranking

In [15]:
# https://github.com/BindiChen/machine-learning/blob/main/data-analysis/031-pandas-multiIndex/multiindex-selection.ipynb
health_ranking = pd.read_excel(os.path.join(raw_dataset, '2020 County Health Rankings Data - v2.xlsx'), sheet_name=None, header=[0, 1])
ranked_measure = health_ranking['Ranked Measure Data']
ranked_measure.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Premature death  \
                FIPS              State             County          Deaths   
0               1000            Alabama                NaN         81791.0   
1               1001            Alabama            Autauga           791.0   
2               1003            Alabama            Baldwin          2967.0   
3               1005            Alabama            Barbour           472.0   
4               1007            Alabama               Bibb           471.0   

                                                                          \
  Years of Potential Life Lost Rate  95% CI - Low 95% CI - High Quartile   
0                       9942.794666   9840.535949  10045.053384      NaN   
1                       8128.591190   7283.340731   8973.841649      1.0   
2                       7354.122530   6918.554270   7789.690790      1.0   
3                      10253.573403   8782.217281  11724.929524      2.0   
4                      11977.539484  10344.064842  13611.014126      3.0   

                                                  \
  YPLL Rate (AIAN) YPLL Rate (AIAN) 95% CI - Low   
0              NaN                           NaN   
1              NaN                           NaN   
2              NaN                           NaN   
3              NaN                           NaN   
4              NaN                           NaN   

                                                    \
  YPLL Rate (AIAN) 95% CI - High YPLL Rate (Asian)   
0                            NaN               NaN   
1                            NaN               NaN   
2                            NaN               NaN   
3                            NaN               NaN   
4                            NaN               NaN   

                                                                  \
  YPLL Rate (Asian) 95% CI - Low YPLL Rate (Asian) 95% CI - High   
0                            NaN                             NaN   
1                            NaN                             NaN   
2                            NaN                             NaN   
3                            NaN                             NaN   
4                            NaN                             NaN   

                                                    \
  YPLL Rate (Black) YPLL Rate (Black) 95% CI - Low   
0               NaN                            NaN   
1      10201.389930                    8123.819140   
2       9890.505276                    8388.547243   
3      12422.166370                   10131.477999   
4      13085.045008                    9430.549497   

                                                        \
  YPLL Rate (Black) 95% CI - High YPLL Rate (Hispanic)   
0                             NaN                  NaN   
1                    12278.960720                  NaN   
2                    11392.463309          3570.475264   
3                    14712.854740                  NaN   
4                    17687.179576                  NaN   

                                                                        \
  YPLL Rate (Hispanic) 95% CI - Low YPLL Rate (Hispanic) 95% CI - High   
0                               NaN                                NaN   
1                               NaN                                NaN   
2                       2372.554883                        5160.330872   
3                               NaN                                NaN   
4                               NaN                                NaN   

                                                    \
  YPLL Rate (White) YPLL Rate (White) 95% CI - Low   
0               NaN                            NaN   
1       7885.665370                    6923.632080   
2       7435.761186                    6940.450728   
3       8140.174177                    6329.263997   
4      12241.384199                   10344.086120   

                 

In [16]:
ranked_measure = ranked_measure.loc[:, 
    [
        ('Unnamed: 0_level_0','FIPS'),
        ('Unnamed: 2_level_0','County'),
        ('Premature death','Years of Potential Life Lost Rate'),
        ('Poor or fair health', '% Fair or Poor Health'),
        ('Adult smoking', '% Smokers'),
        ('Adult obesity', '% Adults with Obesity'),
        ('Primary care physicians', 'Primary Care Physicians Rate'),
        ('Flu vaccinations', '% Vaccinated'),
        ('Unemployment', '% Unemployed'),
        ('Air pollution - particulate matter', 'Average Daily PM2.5'),
        ('Severe housing problems', '% Severe Housing Problems')
    ]
]
ranked_measure.columns = ranked_measure.columns.get_level_values(1)
ranked_measure = ranked_measure[~ranked_measure['County'].isna()].drop('County', axis=1)
missing_percentage(ranked_measure)

Years of Potential Life Lost Rate    9.33
Primary Care Physicians Rate         4.68
Average Daily PM2.5                  1.08
% Vaccinated                         0.57
% Unemployed                         0.03
FIPS                                 0.00
% Fair or Poor Health                0.00
% Smokers                            0.00
% Adults with Obesity                0.00
% Severe Housing Problems            0.00
dtype: float64

In [17]:
ranked_measure.rename({'% Vaccinated': '% Flu Vaccinated'}, axis=1, inplace=True)
write_clean(ranked_measure, 'Health rank measure.csv')

In [18]:
additional_measure = health_ranking['Additional Measure Data']
# for year 2020
additional_measure = additional_measure.loc[:, 
    [
        ('Unnamed: 0_level_0','FIPS'),
        ('Diabetes prevalence','% Adults with Diabetes'),
        ('Demographics', '% Black'),
        ('Demographics', '% American Indian & Alaska Native'),
        ('Demographics', '% 65 and over')
    ]
]

# for year 2021, 2022
# additional_measure = additional_measure.loc[:, 
#     [
#         ('Unnamed: 0_level_0','FIPS'),
#         ('Diabetes prevalence','% Adults with Diabetes'),
#         ('% Non-Hispanic Black', '% Black'),
#         ('% American Indian & Alaska Native', '% American Indian & Alaska Native'),
#         ('% 65 and older', '% 65 and Over')
#     ]
# ]

additional_measure.columns = additional_measure.columns.get_level_values(1)
write_clean(additional_measure, 'Health additional measure.csv')

## Test data
The “County - Testing Counts - CDC” file from [US Covid Atlas (theuscovidatlas.org)](https://theuscovidatlas.org/download)

In [36]:
tests = read_raw('covid_testing_cdc.csv')
tests = tests.fillna(0).melt(
    id_vars=['fips_code'], 
    var_name="Date", 
    value_name="Tests"
).reset_index(drop=True)
tests.rename({'fips_code':'FIPS'}, axis=1, inplace=True)
# tests = tests[tests['Date']>=start_date]

tests = tests[['Date', 'FIPS', 'Tests']].pivot_table(
    values='Tests',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)

write_clean(tests, 'Testing.csv')

## Spread
Covid cumulative cases from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map)

In [57]:
df = read_raw('covid_confirmed_usafacts.csv')
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)

# drop invalid rows
df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df = df.fillna(0)

df.head()

,FIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2

In [58]:
date_columns = [col for col in df.columns if valid_date(col)]
advanced_days = 13
df_advanced = df[date_columns].shift(periods=advanced_days, axis=1, fill_value=0)

df_spread = 1 - df_advanced.div(df[date_columns], axis=1)

df_spread.fillna(0, inplace=True)
df_spread = df_spread.replace(np.inf, 0).replace(-np.inf, 0)

num = df_spread._get_numeric_data()
num[num < 0] = 0

df_spread['FIPS'] = df['FIPS']
df_spread = df_spread[['FIPS']+ date_columns]

df_spread[date_columns] = MinMaxScaler().fit_transform(df_spread[date_columns])
df_spread.head()

,FIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2

In [59]:
write_clean(df_spread, 'Disease Spread.csv')